In [ ]:
import re
import os
import ast
import json
import pandas as pd
import numpy as np
from collections import defaultdict
import itertools
from collections import Counter
import glob
import random
import shutil
from tqdm.notebook import tqdm
from scipy.spatial import distance as dist
import cv2
from __future__ import print_function
from imutils import perspective
from imutils import contours
import imuti ls
import argparse

In [ ]:
def read_pages(text):
    url, list_dict = text.split('\t')
    list_dict = ast.literal_eval(
        list_dict.replace('\n', '')
        .replace('false', 'False')
        .replace('true', 'True')
    )
    return [url, list_dict]

# Split validate imgs

In [ ]:
!unzip -q "/content/drive/MyDrive/Study/KLTN/data/NomNaOCR/Pages.zip" -d "/tmp"

In [ ]:
PATH_IMGS = []
with open('/tmp/Pages/Validate.txt', 'r') as f:
    PATH_IMGS.extend([label.split('\t')[0] for label in f.readlines()])
PATH_IMGS[0:5]

['Luc Van Tien/imgs/nlvnpf-0059-078.jpg',
 'Luc Van Tien/imgs/nlvnpf-0059-079.jpg',
 'Luc Van Tien/imgs/nlvnpf-0059-075.jpg',
 'Luc Van Tien/imgs/nlvnpf-0059-072.jpg',
 'Luc Van Tien/imgs/nlvnpf-0059-058.jpg']

In [ ]:
set(map(lambda x: x.split('/imgs/')[0], PATH_IMGS))

{'DVSKTT-1 Quyen thu',
 'DVSKTT-2 Ngoai ky toan thu',
 'DVSKTT-3 Ban ky toan thu',
 'DVSKTT-4 Ban ky thuc luc',
 'DVSKTT-5 Ban ky tuc bien',
 'Luc Van Tien',
 'Tale of Kieu 1866',
 'Tale of Kieu 1871',
 'Tale of Kieu 1872'}

In [ ]:
LABEL_VAL = []
DATA_VAL = []
LIST_POEM = [
  'Luc Van Tien', 
  'Tale of Kieu 1866', 
  'Tale of Kieu 1871', 
  'Tale of Kieu 1872'
]

LIST_PROSE = [
  'DVSKTT-1 Quyen thu',
  'DVSKTT-2 Ngoai ky toan thu',
  'DVSKTT-3 Ban ky toan thu',
  'DVSKTT-4 Ban ky thuc luc',
  'DVSKTT-5 Ban ky tuc bien'
]

with open('/tmp/Pages/Tool/Validate.txt', 'r') as f:
     list_data = f.readlines()
     for data in list_data:
          url, list_dict = read_pages(data)
          LABEL_VAL.append(url)
          DATA_VAL.append(list_dict)

In [ ]:
df_map_label = pd.read_csv('/content/drive/MyDrive/Study/KLTN/data/map_gt_filename.csv')
dict_map_label = {file_name: gt for file_name, gt in df_map_label.values}

In [ ]:
def order_points_old(pts):
	rect = np.zeros((4, 2), dtype="float32")
	s = pts.sum(axis=1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	diff = np.diff(pts, axis=1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	return rect

  
def order_points(pts):
    xSorted = pts[np.argsort(pts[:, 0]), :]

    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]

    return np.array([tl, tr, br, bl], dtype="float32")

In [ ]:
new = list([int(num) for num in order_points(np.array([[121, 128], [172, 69], [164, 62], [129, 134]])).flat])

In [ ]:
old = list([int(num) for num in order_points_old(np.array([[121, 128], [172, 69], [164, 62], [129, 134]])).flat])

In [ ]:
def extract_data(list_data):
    results = []
    for data in list_data:
        list_points = list([int(num) for num in order_points_old(np.array(data['points'])).flat])
        str_point = ", ".join([str(num) for num in list_points])
        results.append(f'{str_point}, {data["transcription"]}')
    return '\n'.join(results)

In [ ]:
list_all = []
list_poem = []
list_prose = []
dict_book = defaultdict(list)

list_all_gt = []
list_poem_gt = []
list_prose_gt = []
dict_book_gt = defaultdict(list)

for label, dict_data in zip(LABEL_VAL, DATA_VAL):
    book_name, file_name = label.split('/imgs/')
    if book_name in LIST_POEM:
        list_poem.append([label.replace(f'{book_name}/', '')
                              .replace(f'{file_name}', f'{book_name}_{file_name}'), dict_data])
        list_poem_gt.append([f'{book_name}_{file_name}'.replace('jpg', 'txt'), extract_data(dict_data)])

    if book_name in LIST_PROSE:
        list_prose.append([label.replace(f'{book_name}/', '')
                               .replace(f'{file_name}', f'{book_name}_{file_name}'), dict_data])
        list_prose_gt.append([f'{book_name}_{file_name}'.replace('jpg', 'txt'), extract_data(dict_data)])

    dict_book[book_name].append([label.replace(f'{book_name}/', '')
                                     .replace(f'{file_name}', f'{book_name}_{file_name}'), dict_data])
    dict_book_gt[book_name].append([f'{book_name}_{file_name}'.replace('jpg', 'txt'), extract_data(dict_data)])
    
    list_all.append([label.replace(f'{book_name}/', '')
                         .replace(f'{file_name}', f'{book_name}_{file_name}'), dict_data])
    list_all_gt.append([f'{book_name}_{file_name}'.replace('jpg', 'txt'), extract_data(dict_data)])

In [ ]:
list_all_gt[0]

['Luc Van Tien_nlvnpf-0059-057.txt',
 '363, 170, 396, 170, 396, 383, 363, 383, 喂傕𤜯扒𠊛芇\n363, 421, 398, 421, 398, 676, 363, 676, 𠊛芇𤜯扒\U000f0761𠓨底低\n323, 173, 358, 173, 358, 381, 323, 381, 吁𩲉吁魄老尼\n326, 421, 361, 421, 361, 680, 326, 680, 迻猪𡮇姅扽核𧵆耒\n286, 173, 319, 173, 319, 383, 286, 383, 雲仙𠬠聀𣃣耒\n288, 422, 323, 422, 323, 680, 288, 680, 侈𦖑㗂呐亘𡎢𨱽呌\n246, 172, 281, 172, 281, 382, 246, 382, 老樵𦖑㗂埃呌\n251, 421, 283, 421, 283, 681, 251, 681, 真𨃝庒恪如𨅹𨖲核\n204, 172, 240, 172, 240, 386, 204, 386, 𣃣𨁏𣃣𨀺𣃣𪡔\n211, 422, 248, 422, 248, 684, 211, 684, 畨尼莫沛禍魔害𫅷\n168, 174, 200, 174, 200, 383, 168, 383, 仙浪碎本𠊚些\n173, 423, 208, 423, 208, 685, 173, 685, 爲蒸莫𮥷買𦋦世尼\n129, 174, 164, 174, 164, 383, 129, 383, 語𥉩量𢌌高苔\n132, 423, 169, 423, 169, 684, 132, 684, 救碎𠬠𣅶恩\U000f07f4再生\n94, 174, 126, 174, 126, 385, 94, 385, 老樵欺意歇驚\n97, 424, 129, 424, 129, 687, 97, 687, 吏𧵆連哙事情買咍\n57, 175, 92, 175, 92, 382, 57, 382, 郑𢚸傷細𠊚𣦍\n60, 425, 93, 425, 93, 687, 60, 687, 尼又迻老𢩮𢬣𧗱茄\n26, 174, 54, 174, 54, 386, 26, 386, 仙浪𩛜㐌㤕車\n25, 427, 56, 427, 56, 689, 25

In [ ]:
for file_name, data in list_all_gt:
    with open(os.path.join("/tmp/validate_all/truth", f'{dict_map_label[file_name]}.txt'), 'w+', encoding='utf-8') as f:
        f.write(data)


for file_name, data in list_poem_gt:
    with open(os.path.join("/tmp/validate_poem/truth", f'{dict_map_label[file_name]}.txt'), 'w+', encoding='utf-8') as f:
        f.write(data)


for file_name, data in list_prose_gt:
    with open(os.path.join("/tmp/validate_prose/truth", f'{dict_map_label[file_name]}.txt'), 'w+', encoding='utf-8') as f:
        f.write(data)


for dir_name, values in dict_book_gt.items():
    for file_name, data in values:
        with open(os.path.join(f"/tmp/validate/{dir_name}/truth", f'{dict_map_label[file_name]}.txt'), 'w+', encoding='utf-8') as f:
            f.write(data)

In [ ]:
def write_txt(path, data):
    with open(path, 'w+', encoding='utf-8') as f:
        f.write(data)


def list2json(data):
    results = []
    for d in data:
        path, value = d
        value = json.dumps(value, ensure_ascii=False).encode('utf-8').decode()
        results.append(f'{path}\t{value}')
    return '\n'.join(results)

In [ ]:
len(list_poem)

102

In [ ]:
len(list_prose)

492

In [ ]:
list2json(list_poem)

In [ ]:
write_txt('/tmp/validate_all/Validate.txt', list2json(list_all))
write_txt('/tmp/validate_poem/Validate.txt', list2json(list_poem))
write_txt('/tmp/validate_prose/Validate.txt', list2json(list_prose))

for key, values in dict_book.items():
    write_txt(f'/tmp/validate/{key}/Validate.txt', list2json(list(values)))

In [ ]:
for idx, path in enumerate(PATH_IMGS):
    target_poem = ''
    target_prose = ''
    book_name = path.split('/imgs/')[0]
    source = os.path.join('/tmp/Pages', path)
    dir_name, file_name = path.replace('/imgs', '').split("/")
    target = os.path.join('/tmp/validate', dir_name, 'imgs', f'{dir_name}_{file_name}')
    target_all = os.path.join('/tmp/validate_all', 'imgs', f'{dir_name}_{file_name}')
    if book_name in LIST_POEM:
        target_poem = os.path.join('/tmp/validate_poem', 'imgs', f'{dir_name}_{file_name}')
        shutil.copy(source, target_poem)


    if book_name in LIST_PROSE:
        target_prose = os.path.join('/tmp/validate_prose', 'imgs', f'{dir_name}_{file_name}')
        shutil.copy(source, target_prose)

    shutil.copy(source, target)
    shutil.copy(source, target_all)

In [ ]:
len(os.listdir('/tmp/validate_all/imgs'))

594

In [ ]:
len(os.listdir('/tmp/validate_poem/imgs'))

102

In [ ]:
len(os.listdir('/tmp/validate_prose/imgs'))

492

In [ ]:
len(glob.glob('/tmp/validate/*/imgs/*.jpg'))

594

In [ ]:
os.listdir('/tmp/validate')

['DVSKTT-3 Ban ky toan thu',
 'DVSKTT-1 Quyen thu',
 'Tale of Kieu 1872',
 'DVSKTT-4 Ban ky thuc luc',
 'DVSKTT-2 Ngoai ky toan thu',
 'Luc Van Tien',
 'Tale of Kieu 1866',
 'Tale of Kieu 1871',
 'DVSKTT-5 Ban ky tuc bien']

/tmp/validate_all/truth
/tmp/validate_poem/truth
/tmp/validate_prose/truth
/tmp/validate/DVSKTT-1 Quyen thu/truth
/tmp/validate/DVSKTT-2 Ngoai ky toan thu/truth
/tmp/validate/DVSKTT-3 Ban ky toan thu/truth
/tmp/validate/DVSKTT-4 Ban ky thuc luc/truth
/tmp/validate/DVSKTT-5 Ban ky tuc bien/truth
/tmp/validate/Luc Van Tien/truth
/tmp/validate/Tale of Kieu 1866/truth
/tmp/validate/Tale of Kieu 1871/truth
/tmp/validate/Tale of Kieu 1872/truth


In [ ]:
PATH_RESULT_MODEL = '/content/drive/MyDrive/Study/KLTN/data/validate/results'

In [ ]:
list_general_db = glob.glob(os.path.join(PATH_RESULT_MODEL, '*', 'predict_db.txt'))
list_general_east = glob.glob(os.path.join(PATH_RESULT_MODEL, '*', 'predict_east.txt'))

list_book_db = glob.glob(os.path.join(PATH_RESULT_MODEL, 'book', '*', 'predict_db.txt'))
list_book_east = glob.glob(os.path.join(PATH_RESULT_MODEL, 'book', '*', 'predict_east.txt'))
list_book_east

['/content/drive/MyDrive/Study/KLTN/data/validate/results/book/Luc Van Tien/predict_east.txt',
 '/content/drive/MyDrive/Study/KLTN/data/validate/results/book/Tale of Kieu 1871/predict_east.txt',
 '/content/drive/MyDrive/Study/KLTN/data/validate/results/book/Tale of Kieu 1872/predict_east.txt',
 '/content/drive/MyDrive/Study/KLTN/data/validate/results/book/DVSKTT-1 Quyen thu/predict_east.txt',
 '/content/drive/MyDrive/Study/KLTN/data/validate/results/book/DVSKTT-2 Ngoai ky toan thu/predict_east.txt',
 '/content/drive/MyDrive/Study/KLTN/data/validate/results/book/DVSKTT-3 Ban ky toan thu/predict_east.txt',
 '/content/drive/MyDrive/Study/KLTN/data/validate/results/book/DVSKTT-4 Ban ky thuc luc/predict_east.txt',
 '/content/drive/MyDrive/Study/KLTN/data/validate/results/book/DVSKTT-5 Ban ky tuc bien/predict_east.txt',
 '/content/drive/MyDrive/Study/KLTN/data/validate/results/book/Tale of Kieu 1866/predict_east.txt']

In [ ]:
def paddle2gt(list_label):
    for path in list_label:
        model_name = re.findall('predict_(.*)\.txt', path)[0]
        raws = []
        map_label = {}
        with open(path, 'r', encoding='utf-8') as f:
            raws = f.readlines()
        for raw in raws:
            url, data = read_pages(raw)
            dir_name = re.findall('(.*)\/imgs/.*', url)[0]
            file_name = os.path.basename(url).replace('jpg', 'txt')
            results = extract_data(data).replace(', \n', '\n').strip(', ')
            with open(os.path.join(dir_name, model_name, f'{dict_map_label[file_name]}.txt'), 'w+', encoding='utf-8') as f:
                f.write(results)

In [ ]:
list_general_db = glob.glob(os.path.join(PATH_RESULT_MODEL, '*', 'predict_db.txt'))
paddle2gt(list_general_db)

list_general_east = glob.glob(os.path.join(PATH_RESULT_MODEL, '*', 'predict_east.txt'))
paddle2gt(list_general_east)

list_book_db = glob.glob(os.path.join(PATH_RESULT_MODEL, 'book', '*', 'predict_db.txt'))
paddle2gt(list_book_db)

list_book_east = glob.glob(os.path.join(PATH_RESULT_MODEL, 'book', '*', 'predict_east.txt'))
paddle2gt(list_book_east)

/tmp/validate_all/db
/tmp/validate_poem/db
/tmp/validate_prose/db
/tmp/validate/DVSKTT-1 Quyen thu/db
/tmp/validate/DVSKTT-2 Ngoai ky toan thu/db
/tmp/validate/DVSKTT-3 Ban ky toan thu/db
/tmp/validate/DVSKTT-4 Ban ky thuc luc/db
/tmp/validate/DVSKTT-5 Ban ky tuc bien/db
/tmp/validate/Luc Van Tien/db
/tmp/validate/Tale of Kieu 1866/db
/tmp/validate/Tale of Kieu 1871/db
/tmp/validate/Tale of Kieu 1872/db
/tmp/validate_all/east
/tmp/validate_poem/east
/tmp/validate_prose/east
/tmp/validate/DVSKTT-1 Quyen thu/east
/tmp/validate/DVSKTT-2 Ngoai ky toan thu/east
/tmp/validate/DVSKTT-3 Ban ky toan thu/east
/tmp/validate/DVSKTT-4 Ban ky thuc luc/east
/tmp/validate/DVSKTT-5 Ban ky tuc bien/east
/tmp/validate/Luc Van Tien/east
/tmp/validate/Tale of Kieu 1866/east
/tmp/validate/Tale of Kieu 1871/east
/tmp/validate/Tale of Kieu 1872/east
